In [ ]:
!git clone http://github.com/Zapy67/ATML-PA-4

Cloning into 'ATML-PA-4'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 47 (delta 20), reused 38 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 289.00 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
!git pull http://github.com/Zapy67/ATML-PA-4

From http://github.com/Zapy67/ATML-PA-4
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [ ]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [ ]:
!ls

'ATML - PA 4.pdf'   fed_lib   README.md     task2.ipynb   task4.ipynb
 data		    LICENSE   task1.ipynb   task3.ipynb


#  FedSAM on heterogeneous domains

In [1]:
import torch
import torch.nn as nn

c:\Users\shaya\anaconda3\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from 'd:\\shayann\\Uni_Resources\\Semester_5\\CS-6304\\PA_4\\ATML-PA-4\\fed_lib\\__init__.py'>

In [3]:
!pip install fedlab

In [35]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

<module 'fed_lib.fed_model' from 'd:\\shayann\\Uni_Resources\\Semester_5\\CS-6304\\PA_4\\ATML-PA-4\\fed_lib\\fed_model.py'>

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [44]:
trainset, testset = utils.get_cifar10()

In [19]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [20]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.115 0.067 0.038 0.061 0.09  0.137 0.224 0.106 0.097 0.065]
Entropy of Label Distribution H(P(Y)): 2.0966035640476957
Normalized Entropy of Label Distribution: 0.9105433586046051

Label Distribution (P(Y)): [0.037 0.029 0.243 0.028 0.066 0.035 0.165 0.065 0.194 0.136]
Entropy of Label Distribution H(P(Y)): 1.9407858730619996
Normalized Entropy of Label Distribution: 0.8428725952266113

Label Distribution (P(Y)): [0.112 0.195 0.049 0.279 0.007 0.188 0.009 0.061 0.064 0.036]
Entropy of Label Distribution H(P(Y)): 1.8371828209927195
Normalized Entropy of Label Distribution: 0.7978783614045877



KeyboardInterrupt: 

In [36]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [37]:
client_weights

[0.26754, 0.23378, 0.20014, 0.0871, 0.21144]

In [52]:
partition = "dirichlet"
device = "cpu"
alpha = 0.1
num_clients = 5
fed = fed_model.Federation(num_clients, None, partition, domains, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

In [53]:
client_loaders = fed.client_dataloaders
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [54]:
fed.set_method(fed_methods.FedSAM(sam_rho=0, 
             num_local_steps=len(trainset)/num_clients, 
             client_aggregation_weights= client_weights))

In [ ]:
fed.train(25, lr=0.05, verbose=False)


--- Round 1/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: 2.3030, Test Acc: 10.00%

--- Round 2/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 2:


FedSAM  | Test Loss: 2.3030, Test Acc: 10.00%

--- Round 3/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5


In [127]:
fed.clients[0]

SmallCNN(
  (backbone): Sequential(
    (0): SmallConvBlock(in_channels=3, out_channels=32)
    (1): SmallConvBlock(in_channels=32, out_channels=64)
    (2): AvgPool2d(kernel_size=1, stride=1, padding=0)
  )
  (task_head): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)